In [1]:
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization

Using TensorFlow backend.


In [3]:
#Read data from csv
train_data = pd.read_csv('fmnist/fashion-mnist_train.csv')
test_data = pd.read_csv('fmnist/fashion-mnist_test.csv')

In [4]:
#Setting images parameters
img_rows, img_cols = 28, 28
input_shape = (img_rows, img_cols, 1)

X = np.array(train_data.iloc[:, 1:])
y = to_categorical(np.array(train_data.iloc[:,0]))

In [5]:
#Separating train data into train and validation set
X_train, X_val, Y_train, Y_val = train_test_split(X, y, test_size=0.2, random_state=13)

In [6]:
#Load test data into X_test and Y_test
X_test = np.array(test_data.iloc[:, 1:])
Y_test = to_categorical(np.array(test_data.iloc[:, 0]))

#Now preprocessing of data
X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
X_val = X_val.reshape(X_val.shape[0], img_rows, img_cols, 1)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_val = X_val.astype('float32')

X_train /= 255
X_test /= 255
X_val /=255

In [12]:
#Building our CNN using keras

batch_size = 256
n_classes = 10
epochs = 50

model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), activation='relu', kernel_initializer='he_normal', input_shape=input_shape))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(128, (3,3), activation='relu'))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(n_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(), metrics=['accuracy'])

In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 128)         73856     
__________

In [9]:
import tensorflow as tf
tf.reset_default_graph()
tbcallback = keras.callbacks.TensorBoard(log_dir="./Graph", histogram_freq=0, write_graph=True, write_images=True)

In [14]:
#Evaluation of our model
history = model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(X_val,Y_val), callbacks=[tbcallback])

Train on 48000 samples, validate on 12000 samples
Epoch 1/50
48000/48000 [==============================] - 25s - loss: 0.8639 - acc: 0.6766 - val_loss: 0.5336 - val_acc: 0.8032
Epoch 2/50
48000/48000 [==============================] - 21s - loss: 0.5281 - acc: 0.8041 - val_loss: 0.4244 - val_acc: 0.8417
Epoch 3/50
48000/48000 [==============================] - 21s - loss: 0.4520 - acc: 0.8362 - val_loss: 0.3698 - val_acc: 0.8643
Epoch 4/50
48000/48000 [==============================] - 21s - loss: 0.4092 - acc: 0.8520 - val_loss: 0.3351 - val_acc: 0.8767
Epoch 5/50
48000/48000 [==============================] - 21s - loss: 0.3763 - acc: 0.8617 - val_loss: 0.3178 - val_acc: 0.8832
Epoch 6/50
48000/48000 [==============================] - 21s - loss: 0.3615 - acc: 0.8699 - val_loss: 0.3061 - val_acc: 0.8879
Epoch 7/50
48000/48000 [==============================] - 21s - loss: 0.3373 - acc: 0.8777 - val_loss: 0.2972 - val_acc: 0.8902
Epoch 8/50
48000/48000 [==============================

In [15]:
#Accuracy on testing data
score = model.evaluate(X_test, Y_test, verbose=0)

In [16]:
#Print testing data loss and accuracy of our model
print('Loss:',score[0])
print('Accuracy:',score[1])

Loss: 0.205050881004
Accuracy: 0.9248


In [17]:
#save model
model.save('fashion-mnist-cnn.json')
model.save_weights('fashion-mnist-weights.h5')